# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
import seaborn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
# Переведем столбцы с количемтвом минут и количеством сообщений в тип int, потому как в данных столбцах используются целочисленные значения
df['calls'] = df['calls'].astype('int64')
df['messages'] = df['messages'].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


**Вывод**
Тип данных у количества звонков и количества сообщений должен быть целочисленный - корректировка внесена. Объём датасета 3214 записей. Пропусков в данных не обнаружено.

## Разбейте данные на выборки

In [5]:
#Извлечем признаки для обучающей выборки
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

In [6]:
#Разабьём данные на обучающую, тестовую, валидационную выборки.
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345) 

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

print('Размер тренировочной выборки:', features_train.shape) # 75% данных
print('Размер тестовой выборки:', features_test.shape) # 12.5% данных
print('Размер валидационной выборки:', features_valid.shape) #12.5#данных

Размер тренировочной выборки: (2410, 4)
Размер тестовой выборки: (402, 4)
Размер валидационной выборки: (402, 4)


## Исследуйте модели

Воспользуемся тремя основными моделями: дерево решения, случайный лес и логистическая регрессия.

In [7]:
# Проведем исследование модели Дерево решений.

best_model_dt = None
best_depth = 0
best_result = 0

for depth in range(1, 11):    
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predict_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predict_valid)
    if result > best_result:
        best_result = result
        best_model_dt = model
        best_depth = depth
        
        
print('Лучший результат accuracy модели Дерево решений: \n\n', best_result,\
      '\n глубина:', best_depth)

Лучший результат accuracy модели Дерево решений: 

 0.7985074626865671 
 глубина: 3


In [8]:
# Проведем исследование модели класса Случайный лес.

best_model_rf = None
best_depth = 0
best_result = 0
best_est = 0

for est in range(10, 51, 10):
    for depth in range(1, 11):
    
        model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        model.fit(features_train, target_train)
        predict_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predict_valid)
        if result > best_result:
            best_result = result
            best_model_rf = model
            best_depth = depth
            best_est = est
                
print('Лучший результат accuracy модели Случайный лес: \n\n', \
      best_result, '\n глубина:', best_depth, '\n число деревьев', best_est)

Лучший результат accuracy модели Случайный лес: 

 0.8159203980099502 
 глубина: 9 
 число деревьев 20


In [9]:
# Проведем исследование модели класса Логистическая регрессия.

model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
predict_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predict_valid)
          
print('Лучший результат accuracy модели Логистическая регрессия: \n\n', \
      result)

Лучший результат accuracy модели Логистическая регрессия: 

 0.7039800995024875


**Вывод**

Исследование показало, что модель, построенная на алгоритме случайного леса при глубине 9 и 20 деревьях, дает лучший результат (0.815). Но она и самая времязатратная. Довольно быстрая и с чуть худшим результом (0.798) - модель на алгоритме дерева решений при глубине 3. И худший результат (0.703) показала модель на алгоритме логистической регрессии.

## Проверьте модель на тестовой выборке

In [10]:
# Проведем проверку выбранной модели с лучшим результатом (модель на алгоритме случайного леса) на тестовой выборке

predict_test = best_model_rf.predict(features_test)
result = accuracy_score(target_test, predict_test)

print('Результат accuracy выбранной модели на тестовой выборке: \n\n', result)

Результат accuracy выбранной модели на тестовой выборке: 

 0.8059701492537313


**Вывод**

Видим небольшую переобученность модели, т.к. результат на тестовой выборке (0.805) немного хуже, чем на валидационной (0.815). Но это значение accuracy выше заданного 0.75.

## (бонус) Проверьте модели на адекватность

In [11]:
# Выясним соотношение тарифов Ультра и Смарт в целевом признаке датасета

df['is_ultra'].mean()

0.30647168637212197

In [12]:
x = df.drop(['is_ultra'], axis=1)
y = df["is_ultra"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=27)

dummy = DummyClassifier(strategy='most_frequent').fit(x_train, y_train)
dummy_prediction = dummy.predict(x_test)

print('test_score :', accuracy_score(y_test, dummy_prediction))

test_score : 0.6932171748599876


**Вывод**

Тест на адекватность пройден, т.к. DummyClassifier показывает точность 0.69, это почти в точности соответствует распределению классов в таргете: доля пользователей с тарифом «Ультра» - 0.306, «Смарт» - 0.694

**Выводы**

1.В ходе исследования удалось построить модель с максимально большим значением accuracy для классификации целевого признака - подбор тарифа мобильного пользователя.

2.Выбор состоял из трех моделей по алгоритмам классификации: дерево решений. случайный лес и логистическая регрессия.

3.Лучшей стала модель на алгоритме случайного леса с глубиной: 9 и числом деревьев: 20.

4.Доля правильных ответов на тестовой выборке достигла 0.805.